In [2]:
%run ../../qsu.ipynb  # color-printing functions
import numpy as np
from qecsim import paulitools as pt
from qecsim.models.generic import DepolarizingErrorModel, BitFlipErrorModel, PhaseFlipErrorModel
from qecsim.models.planar import PlanarCode, PlanarMWPMDecoder, PlanarMPSDecoder
from TI_MSC import ti_calc_serial_spin_encoded, ti_calc_Parallel_spin_encoded

# initialise models
d = 7
my_code = PlanarCode(d, d)
my_error_model = PhaseFlipErrorModel()
#my_decoder = PlanarMWPMDecoder()
# print models
print(my_code)
print(my_error_model)

PlanarCode(7, 7)
PhaseFlipErrorModel()


In [3]:
# set physical error probability to 10%
error_probability = 0.1
# seed random number generator for repeatability
rng = np.random.default_rng(1111)

# error: random error based on error probability
error = my_error_model.generate(my_code, error_probability, rng)
qsu.print_pauli('error:\n{}'.format(my_code.new_pauli(error)))

In [4]:
# syndrome: stabilizers that do not commute with the error
syndrome = pt.bsp(error, my_code.stabilizers.T)
qsu.print_pauli('syndrome:\n{}'.format(my_code.ascii_art(syndrome)))

In [5]:
# from mps decoder
my_mps_decoder = PlanarMPSDecoder(chi=6)
coset_ps, recoveries = my_mps_decoder.calc_coset_probabilities(
    my_code,
    syndrome,
    error_model=my_error_model,
    error_probability=error_probability)

print('total prob of each coset', coset_ps)
print('Only the first and last are non-zero because only Z errors.')
print('The log of their ratio is:',
      np.log(float(coset_ps[3]) / float(coset_ps[0])))

total prob of each coset (mpf('1.5758944953105232e-11'), mpf('0.0'), mpf('0.0'), mpf('2.6072957444614728e-12'))
Only the first and last are non-zero because only Z errors.
The log of their ratio is: -1.7990945666238425


In [6]:
recovery = recoveries[0].to_bsf()
qsu.print_pauli('recovery:\n{}'.format(my_code.new_pauli(recovery)))

In [7]:
# success iff recovery ^ error commutes with logicals
print(pt.bsp(recovery ^ error, my_code.logicals.T))

[0 0]


In [8]:
# couriosity:recovery ^ error
qsu.print_pauli('recovery ^ error:\n{}'.format(my_code.new_pauli(recovery ^ error)))

In [9]:
ti_calc_Parallel_spin_encoded(recovery,
               error_probability=error_probability,
               d=d,
               num_int_steps=101,
               mcmove_steps_init = 500000, 
               mcmove_steps_for_derivative=1000,
               num_samples=1000)